<a href="https://colab.research.google.com/github/frba/Nanopore_pipeline_colab/blob/main/Pipeline_for_Nanopore_Sequecing_Data_Small_reads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pipeline for Nanopore Sequencing Data

## Step 1 - Upload files and Installing packages

---



1.   Upload your fastq files containing reads (raw data from plasmidsaurus)
2.   Upload the minibar barcode_file.txt - [example_barcode_file](https://liveconcordia.sharepoint.com/:t:/t/GenomeFoundryTeam/EQsYiSKVWo5Oli4UnBnkIEABReHpAAEuzrEk2Cypi6QlMw?e=1tqfym)
3.   Upload fasta file containing the expected sequences - [example_fasta_template](https://liveconcordia.sharepoint.com/:u:/t/GenomeFoundryTeam/Eb9_w8lNcoFEmh9iUuME76EBpCtpj6ZHnsJcx9UtqFyuOQ?e=4SjzwQ)
4.   Upload a [Echo transfer.csv](https://liveconcordia.sharepoint.com/:x:/t/GenomeFoundryTeam/EYVzHEZPGb9DqxJDN56S_6kBc1_-whiPIm1jkmkiuh-U3A?e=tD3xl6) file (input file used in Echo)

```markdown
5.   **Run the Step 1, it will crash and restart the runtime. Run it again and the following cells.**
```

In [ ]:
version = !conda --version
if version[0] == "/bin/bash: line 1: conda: command not found":
    !pip install -q condacolab
    import condacolab
    condacolab.install()
!conda config --add channels bioconda
!conda install -c conda-forge -c bioconda mmseqs2
!conda install -c bioconda blast
!conda install flye
!conda install -c bioconda seqkit
%pip install biopython
%pip install edlib

import glob
import os
import pandas as pd
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq
import subprocess
import shutil
import zipfile

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested pack

## Step 1.1 - Installing Tools: SPOA, Filtlong, Minibar

1.   Installing SPOA (De novo assembler tool),
2.   Filtlong (Filtering tool), and  
3.   Minibar (Demultiplex tool)

In [ ]:
!git clone https://github.com/rvaser/spoa
%cd ./spoa
!cmake -B build -DCMAKE_BUILD_TYPE=Release
!make -C build
%cd ..

!curl https://raw.githubusercontent.com/calacademy-research/minibar/master/minibar.py > minibar.py
!chmod +x minibar.py

!git clone https://github.com/rrwick/Filtlong.git
%cd ./Filtlong
!make -j
%cd ..

Cloning into 'spoa'...
remote: Enumerating objects: 1360, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1360 (delta 213), reused 197 (delta 187), pack-reused 1098 (from 1)
Receiving objects: 100% (1360/1360), 526.51 KiB | 16.45 MiB/s, done.
Resolving deltas: 100% (905/905), done.
/content/spoa
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning (dev) at /usr/local/lib/python3.10/dist-packages/cmake/data/share/cmake-3.30/Modules/FetchContent.cmake:1953 (message):
  Calling FetchContent_Populate(bioparser) is deprecated, call
  FetchContent_MakeAvailable(bioparser) instead.  Policy CMP0169 can be set
  to OLD to allow FetchContent_Populate(bioparser) to be called directly for
  now, but the ability to

##Step 2 - Set variable names (**Needs input from User**)

---



From the uploaded files, set the filename to each variable below:

*   barcodes_file  (a .txt file with the sequence of pair of barcodes).
*   template_file (a fasta file with the expected sequences).

In [ ]:
# prompt: combine multiple fastq files in one unique file
# Assuming ONLY your raw fastq files are in the current directory

!cat *.fastq >> merged_fastq_file.fastq # merge all raw data in one unique fastq file
basecalled_file = 'merged_fastq_file.fastq' # do not modify this filename

barcode_file = 'barcodes.txt' #<--- Rename input file
template_file = 'genes.fasta' #<--- Rename input file

##Step 3 - Filtering with Filtlong (Needs input from User)

---




*   Set **min** and **max** length of the reads

_Filtlong will remove reads too small or too long from the raw data and return a file called filtered.fastq.gz_

In [ ]:
!Filtlong/bin/filtlong --min_length 400 --max_length 800 {basecalled_file} | gzip > filtered.fastq.gz

##Step 4 - Demultiplexing with Minibar

---



_Minibar will use the barcode.txt file + filtered.fastq.gs to demux the reads in bins listed in barcode.txt file_

In [ ]:
!./minibar.py -e 4 -F -T -M 1 -P demux_minibar_1_ {barcode_file} filtered.fastq.gz

!zip -R 'demultiplexed.zip' 'demux_minibar*' -qq
!zip -R 'demultiplexed.zip' {barcode_file} -qq

##Step 5 - Stats information about the demultiplexed reads


###Step 5.1 Count reads per barcode
_Returns a file called count_barcodes.csv with a list of barcodes and number of reads in each._

In [ ]:
index = pd.read_csv(barcode_file, delimiter='\t', index_col=0).index
headers = [1,2,3,4]
data = dict()
for run in headers[:]:
    data[run] = dict()
    prefix = f'demux_minibar_{run}_'
    file_list = glob.glob(f'{prefix}*')
    for file in file_list:
        sample=Path(file).stem[file.find(prefix)+len(prefix):]
        count = len(list(SeqIO.parse(file, format='fastq')))
        data[run][sample] = count

pd.DataFrame(data).to_csv('count_barcodes.csv')

### Step 5.2 Quality of reads by bin (Optional)
_Returns a file called quality_barcodes.csv with a list of barcodes and number of reads, and quality information (Q20%, Q30%, average quality) and more._

filename            | format | type |... | Q20(%)|... |
--------------------|--------|------|----|-------|----|
demux_FP1-RP1.fastaq| FASTQ  | DNA  |... | 88.21 |... |
demux_FP1-RP2.fastaq| FASTQ  | DNA  |... | 96.79 |... |
demux_FP1-RP3.fastaq| FASTQ  | DNA  |... | 79.44 |... |

In [ ]:
import subprocess
import pandas as pd

def calculate_and_merge_quality():
    """Calculates quality of fastq files and merges the output into quality_barcodes.csv."""
    seqkit_cmd = f"seqkit stats demux_minibar_*.fastq -a"
    result = subprocess.run(seqkit_cmd, shell=True, capture_output=True, text=True)

    # Split the output into lines and process each line
    lines = result.stdout.strip().split('\n')

    # Extract header and data
    header = lines[0].split()
    data = [line.split() for line in lines[1:]]

    # Create DataFrame
    df = pd.DataFrame(data, columns=header)

    # Save DataFrame to CSV
    df.to_csv('quality_barcodes.csv', index=False)

calculate_and_merge_quality()

###5.3 Quality of each read in a demux barcode pair (Optional)
_Returns a file called quality_reads.csv with a list of all reads in each barcode and average quality information._

filename            | seqid   | qual
--------------------|---------|---------
demux_FP1-RP1.fastaq| 974_ABC | 25.89
demux_FP1-RP2.fastaq| 128_CDB | 30.01
demux_FP1-RP3.fastaq| 579_CDB | 18.10

In [ ]:
import io

def run(fastq_file, references, i, j):
    seqkit_cmd = f"seqkit fx2tab {fastq_file} -n -q"

    # Run the seqkit command and capture the output
    result = subprocess.run(seqkit_cmd, shell=True, capture_output=True, text=True)

    # Convert output to DataFrame and add filename column
    df = pd.read_csv(io.StringIO(result.stdout), sep='\t', header=None, names=['seq', 'qual'])
    df.insert(0, 'file', fastq_file)  # Insert filename in the first column

    # Append DataFrame to quality_reads.csv
    if not os.path.isfile('quality_reads.csv'):
        df.to_csv('quality_reads.csv', index=False, mode='w')
    else:
        df.to_csv('quality_reads.csv', index=False, header=False, mode='a')


for i in range(1, 10):  # Changed to range(1, 2) to iterate once
  for j in range(1, 10):  # Changed to range(1, 2) to iterate once
    reads_path = f"demux_minibar_1_FP{i}-RP{j}.fastq"
    if os.path.exists(reads_path):
        run(reads_path, template_file, i, j)

##Step 6 - De Novo Consensus with SPOA

---
_SPOA will create a de novo consensus sequence for each pair of barcode._
**bold text**
_A zipfile: assembled.zip will have all consensus created._



In [ ]:
files = glob.glob('demux_minibar_*.fastq')
for file in files:
    cmd = f'./spoa/build/bin/spoa {file} > {file.replace("fastq", "fasta")}'
    !{cmd}

#Compress output files in assembled.zip
!zip -R 'assembled.zip' 'demux_*.fasta' -qq

##Step 7 - Polishing consensus with Flye (Needs User input)

---


*   Variable **estimated_size** needs to be updated!!


_Fasta files with consensus will be polished by Flye and return a file *_polished.fasta and a zipfile polished.zip_

In [ ]:
for i in range(1, 21):
  for j in range(1, 21):
    reads_path = f"demux_minibar_1_FP{i}-RP{j}.fastq" #Filtered and demultiplexed reads
    write_path = f"flye_output_FP{i}-RP{j}" #Folder name
    polish_target = f"demux_minibar_1_FP{i}-RP{j}.fasta" #Output file from SPOA
    estimated_size = 600 #<---- PLEASE, CHECK THE ESTIMATED LENGTH!

    if os.path.exists(reads_path):
      flye_command = f"flye --nano-hq {reads_path} --out-dir {write_path} --genome-size {estimated_size} --meta --keep-haplotypes --polish-target {polish_target}"

      try:
          # Run the flye command, capture stderr, and check for errors
          result = subprocess.run(flye_command, shell=True, check=True, capture_output=True, text=True)
          print(result.stdout)  # Print standard output if successful
      except subprocess.CalledProcessError as e:
          print(f"Error running Flye: {e}")
          print(f"Flye stderr output:\n{e.stderr}")  # Print standard error for debugging

      #Move the polished file once it has been created
      !mv "{write_path}/polished_1.fasta" "{polish_target.replace(".fasta", "_polished.fasta")}"
      print(f"Polished file: {polish_target}")

# Delete all folders starting with "flye_output" and their contents
for folder_name in glob.glob("flye_output*"):
    shutil.rmtree(folder_name)

# Delete files ending with *.fai
for file in glob.glob("*.fai"):
    os.remove(file)

##Step 8 - Rename seqid with filename

_After running SPOA the seqid in the fasta files is renamed to: 'consensus'._

_To keep the trackability of the reads, it needs to be renamed with the filename._

_This step will rename all *_polished.fasta's seqid with their filename._

In [ ]:
files = glob.glob('*_polished.fasta')

for file in files:
    name = file.replace('.fasta', '')
    record = SeqIO.read(file, 'fasta')
    record.id = name
    record.name = name
    record.description = 'filtered with Filtlong, assembled with SPOA and polish with Flye'
    SeqIO.write(record, file, 'fasta')

# Create a zip file named 'polished.zip'
with zipfile.ZipFile('polished.zip', 'w') as zipf:
    # Find all files ending with '_polished.fasta'
    for file in glob.glob("*_polished.fasta"):
        # Add each file to the zip archive
        zipf.write(file)
        print(f"Added to zip: {file}")

##Step 9 - Alignment Tools

###Step 9.1 Run MMSeqs (Sequence Alignment)

_Three files are outputed in this step:_


*   AlignResult.tsv (All alignment combinations)
*   TopAlignments_bygene (Drop duplicates of templates and keep the highest scores).
*   TopAlignments_bybarcode (Drop duplicates of barcodes and keep only the highest alignment scores).



In [ ]:
def run_mmseq(queries, references):
    for query in queries:
        !cat {query} >> mmseq_query.fa # merge all *_polished.fasta in one unique file to use it as input for mmseqs2

    # Run mmseqs
    mmseq_cmd = f"mmseqs easy-search mmseq_query.fa {references} AlignResult.tsv tmp --search-type 3"
    !{mmseq_cmd}
    all_hits = pd.read_csv('AlignResult.tsv', index_col=None, sep='\t', header=None)
    columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    all_hits.columns = columns

    # Drop duplicates of templates
    top_hits = all_hits.sort_values(by='bitscore', ascending=False).drop_duplicates(subset='sseqid')
    top_hits.to_csv('TopAlignments_bygene.tsv', sep='\t', index=False)

    # Drop duplicates of barcodes
    top_hits = all_hits.sort_values(by='bitscore', ascending=False).drop_duplicates(subset='qseqid')
    top_hits.to_csv('TopAlignments_bybarcode.tsv', sep='\t', index=False)

run_mmseq(glob.glob('*_polished.fasta'), template_file)

In [ ]:
pd.read_csv('TopAlignments.tsv', index_col=None, sep='\t')

###Step 9.2 - Run NCBI Blastn

_returns a called blast_results.tsv_

In [ ]:
def run_blastn(queries, references):

    if not os.path.exists("mmseq_query.fa"):
      for query in queries:
          !cat {query} >> mmseq_query.fa # merge all *_polished.fasta in one unique file to use it as input for blastn

    # Create a database file with the templates
    blastdb_cmd = f"makeblastdb -in {template_file} -out blastdb -dbtype nucl"
    !{blastdb_cmd}

    # Run BlastN and output a file: blast_results.tsv
    blast_cmd = f"blastn -query mmseq_query.fa -db blastdb -out blast_results.tsv -outfmt '6 std qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore sseq'"
    !{blast_cmd}

run_blastn(glob.glob('*_polished.fasta'), template_file)

## Step 10 - Add extra info in the Top Alignment files

###Step 10.1 - Add number of reads in TopAlignments

In [ ]:
import pandas as pd

def add_read_count(top_alignments_file):
    """
    Adds read count to the TopAlignments file.

    Args:
        top_alignments_file: Path to the TopAlignments file (tsv).
    """
    # Load the TopAlignments and count_barcodes files
    top_alignments = pd.read_csv(top_alignments_file, sep='\t')
    count_barcodes = pd.read_csv('count_barcodes.csv', index_col=0)

    # Select the first two columns from count_barcodes
    count_barcodes_subset = count_barcodes[['1']]

    # Extract the barcode from the TopAlignments 'qseqid' column
    top_alignments['barcode'] = top_alignments['qseqid'].str.extract(r'demux_minibar_1_(.*)_polished')

    # Merge the two DataFrames based on the barcode
    merged_df = pd.merge(top_alignments, count_barcodes_subset, left_on='barcode', right_index=True, how='left')

    # Rename the merged column to 'read_count'
    merged_df = merged_df.rename(columns={'1': 'read_count'})

    # Reorder columns to place the new columns at the beginning
    cols = ['barcode', 'read_count'] + [col for col in merged_df.columns if col not in ['barcode', 'read_count']]
    merged_df = merged_df[cols]

    # Save the updated TopAlignments file
    merged_df.to_csv(top_alignments_file, sep='\t', index=False)

# Call the function for both files
add_read_count('TopAlignments_bybarcode.tsv')
add_read_count('TopAlignments_bygene.tsv')

###Step 10.2 - Add Plate Well information in Top Alignments


In [ ]:
# prompt: Looking into a .csv file called *echo_transfer
# search the barcode (FP1-RP1) split them by -
# And search in the csv file that looks like this:
# Sample_ID, Sample Name, Source Plate, Source Plate Name, Source Well, Destination Plate Name, Destination Well, Volume,
# pY108_FP1_1, pY108_FP_1, GF0001254, MOTY_OL_001, A1, GF0001264, GF0001264, A1, 25
# pY108_RP1_1, pY108_RP_1, GF0001254, MOTY_OL_001, C1, GF0001264, GF0001264, A1, 25
# pY108_FP1_1, pY108_FP_1, GF0001254, MOTY_OL_001, A1, GF0001264, GF0001264, A2, 25
# pY108_RP2_1, pY108_RP_1, GF0001254, MOTY_OL_001, C1, GF0001264, GF0001264, A2, 25
# The barcode FP1-RP1 will be separated like this: FP1 and RP1 will share the same Destination Well + Destination Plate Name.
# For each barcode in ToAlignments_2.tsv check in the *echo_transfer* the Destination Plate Name and Destination Well and add the information in the TopAlignments_2.tsv

import pandas as pd

def add_plate_well(top_alignments_file):

  # Load the TopAlignments_2.tsv file
  top_alignments = pd.read_csv(top_alignments_file, sep='\t')

  # Load the echo_transfer.csv file
  echo_transfer = pd.read_csv('409_echo_echo_transfer_draw.csv')

  # Function to extract FP and RP barcodes from qseqid
  def extract_barcodes(barcode):
    fp_barcode = barcode.split('-')[0]
    rp_barcode = barcode.split('-')[1]
    return fp_barcode, rp_barcode

  # Add new columns for Destination Plate Name and Destination Well
  top_alignments['Destination Plate Name'] = None
  top_alignments['Destination Well'] = None

  # Iterate through the TopAlignments_2.tsv file
  for index, row in top_alignments.iterrows():
    fp_barcode, rp_barcode = extract_barcodes(row['barcode'])

    if fp_barcode and rp_barcode:
      # Search for matching barcodes in echo_transfer.csv
      matching_rows_fp = echo_transfer[echo_transfer['Sample Name'].str.contains(fp_barcode)]
      matching_rows_rp = echo_transfer[echo_transfer['Sample Name'].str.contains(rp_barcode)]

      # Check if both barcodes have matches
      if not matching_rows_fp.empty and not matching_rows_rp.empty:
        # Find common Destination Plate Name and Destination Well
        common_rows = pd.merge(
          matching_rows_fp[['Destination Plate Name', 'Destination Well']],
          matching_rows_rp[['Destination Plate Name', 'Destination Well']],
          on=['Destination Plate Name', 'Destination Well'],
          how='inner'
        )

        if not common_rows.empty:
          # Get the first common row (assuming there's only one)
          first_common_row = common_rows.iloc[0]
          top_alignments.at[index, 'Destination Plate Name'] = first_common_row['Destination Plate Name']
          top_alignments.at[index, 'Destination Well'] = first_common_row['Destination Well']

  # Reorder columns to place Destination Plate Name and Destination Well at the beginning
  cols = ['Destination Plate Name', 'Destination Well'] + [col for col in top_alignments.columns if col not in ['Destination Plate Name', 'Destination Well']]
  top_alignments = top_alignments[cols]

  # Save the updated TopAlignments.tsv file
  top_alignments.to_csv(top_alignments_file, sep='\t', index=False)


# Call the function for both files
add_plate_well('TopAlignments_bybarcode.tsv')
add_plate_well('TopAlignments_bygene.tsv')

## Step 11 - Clean up temp files

In [ ]:
# Delete all *.fasta files after adding them to the zip
for file in glob.glob("*.fasta"):
    os.remove(file)
    print(f"Deleted file: {file}")

# Delete all *.fastq files after adding them to the zip
for file in glob.glob("*.fastq"):
    os.remove(file)
    print(f"Deleted file: {file}")

# Delete all blastdb.* files
for file in glob.glob("blastdb.*"):
    os.remove(file)
    print(f"Deleted file: {file}")

# Delete all quality* files
for file in glob.glob("quality*"):
    os.remove(file)
    print(f"Deleted file: {file}")

# Delete all *.stats files
for file in glob.glob("*.stats"):
    os.remove(file)
    print(f"Deleted file: {file}")
